In [12]:
from kafka import KafkaConsumer
from json import loads
from time import sleep
from json import dumps
from kafka import KafkaProducer
import requests
import re
import json

In [13]:
import datetime
import redis
import time

In [14]:
consumer = KafkaConsumer(
    'statics',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='redis_consumer',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [15]:
r = redis.Redis()

In [16]:
def add_twitts_recently_list(twitt):
    r.lpush('twitt_recently_list',twitt['content'])
    r.ltrim('twitt_recently_list',0,99)


In [17]:
def add_hashtags_recently_list(twitt):
    if 'hashtags' not in twitt:
        return
    for hashtag in twitt['hashtags']:
        r.lpush('hashtags_recently_list',hashtag)
    r.ltrim('hashtags_recently_list',0,999)


In [40]:
def last_hour_hashtags(twitt):
    if 'hashtags' not in twitt:
        return
    d = datetime.datetime.strptime(twitt['sendTime'].replace('T',' ').replace('Z',''), '%Y-%m-%d %H:%M:%S')
    d += datetime.timedelta(hours=4,minutes=30) #Correct timezone diff for timezone computation
    timestamp = int(time.mktime(d.timetuple()))
#     print(timestamp)
    for hashtag in twitt['hashtags']:
        tmp = {hashtag:timestamp}
        r.zadd('last_hour_tags',tmp)
    current_ts = time.time()
    r.zremrangebyscore('last_hour_tags', 0, current_ts-3600)

#     r.zremrangebyscore('sorted_twitt_set', 0, current_ts-3600)
#     r.zcount(sorted_twitt_set, current_ts-3600, current_ts)

In [19]:
def extract_date_key(date_str):
    d = datetime.datetime.strptime(date_str.replace('T',' ').replace('Z',''), '%Y-%m-%d %H:%M:%S')
#     ret_str = '%d/%d/%d:%d'
    ret_str = "{0}/{1}/{2} {3}H".format(d.year,d.month,d.day,d.hour)
    return ret_str

In [20]:
def count_twitt(twitt):
    name_key = extract_date_key(twitt['sendTime'])
    if r.incrby(name_key,amount=1) < 2:
        d = datetime.datetime.strptime(twitt['sendTime'].replace('T',' ').replace('Z',''), '%Y-%m-%d %H:%M:%S')
        expire_date = d + datetime.timedelta(days=7)
        expire_timestamp = int(time.mktime(expire_date.timetuple()))
        r.expireat(name_key,expire_timestamp)
        print("Key: \t{0} \t\t\tExpire AT: \t{1}".format(name_key,expire_date))
        

In [21]:
def count_hashtags_twitt(twitt):
    if 'hashtags' not in twitt:
        return
    for hashtag in twitt['hashtags']:
        name_key = extract_date_key(twitt['sendTime'])+' tag:'+hashtag
        if r.incrby(name_key,amount=1) < 2:
            d = datetime.datetime.strptime(twitt['sendTime'].replace('T',' ').replace('Z',''), '%Y-%m-%d %H:%M:%S')
            expire_date = d + datetime.timedelta(days=7)
            expire_timestamp = int(time.mktime(expire_date.timetuple()))
            r.expireat(name_key,expire_timestamp)
            print("Key: \t{0} \t\t\tExpire AT: \t{1}".format(name_key,expire_date))
        

In [ ]:
cnt =0
for message in consumer:
    twitt = message.value
#     print(twitt)
    cnt+=1
    if 'sendTime' not in twitt:
        continue
    count_twitt(twitt)
    count_hashtags_twitt(twitt)
    last_hour_hashtags(twitt)
    add_twitts_recently_list(twitt)
    add_hashtags_recently_list(twitt)
    if cnt % 50 ==0:
        print(cnt)


Key: 	2021/7/17 6H 			Expire AT: 	2021-07-24 06:17:23
Key: 	2021/7/17 6H tag:دی 			Expire AT: 	2021-07-24 06:17:18
Key: 	2021/7/17 6H tag:غصینو 			Expire AT: 	2021-07-24 06:17:07
Key: 	2021/7/17 6H tag:وبملت 			Expire AT: 	2021-07-24 06:17:04
Key: 	2021/7/17 6H tag:شتران 			Expire AT: 	2021-07-24 06:16:55
Key: 	2021/7/17 6H tag:شپنا 			Expire AT: 	2021-07-24 06:13:43
Key: 	2021/7/17 6H tag:کرمان 			Expire AT: 	2021-07-24 06:16:44
Key: 	2021/7/17 6H tag:سبزوا 			Expire AT: 	2021-07-24 06:19:53
Key: 	2021/7/17 5H 			Expire AT: 	2021-07-24 05:15:17
Key: 	2021/7/17 5H tag:دی 			Expire AT: 	2021-07-24 05:15:17
Key: 	2021/7/17 6H tag:وسبحان 			Expire AT: 	2021-07-24 06:19:25
Key: 	2021/7/17 6H tag:فولای 			Expire AT: 	2021-07-24 06:19:25
Key: 	2021/7/17 6H tag:قنیشا 			Expire AT: 	2021-07-24 06:19:21
Key: 	2021/7/17 6H tag:سیتا 			Expire AT: 	2021-07-24 06:19:04
Key: 	2021/7/17 6H tag:ثعمرا 			Expire AT: 	2021-07-24 06:21:28
Key: 	2021/7/17 6H tag:فجر 			Expire AT: 	2021-07-24 06:21:16
Key: 

In [36]:
d = datetime.datetime.strptime('2021-07-16 15:14:19'.replace('T',' ').replace('Z',''), '%Y-%m-%d %H:%M:%S').replace(tzinfo=datetime.timezone.utc)
d += datetime.timedelta(hours=4,minutes=30)
timestamp = int(time.mktime(d.timetuple()))

In [37]:
print(timestamp)

1626448459


In [83]:
current_ts = time.time()
r.zcount('last_hour_tags', 0, current_ts)

14

In [84]:
print(r.zrange('last_hour_tags', 0, -1, withscores=True))

[(b'\xda\xaf\xd8\xb4\xd8\xa7\xd9\x86', 1625921191.0), (b'\xd9\xbe\xd9\x84\xd8\xa7\xd8\xb3\xda\xa9', 1625921264.0), (b'\xd8\xb4\xd9\xbe\xd9\x86\xd8\xa7', 1625921352.0), (b'\xd8\xb4\xd8\xb3\xd8\xaa\xd8\xa7', 1625921364.0), (b'\xd9\x85\xd8\xa7\xd8\xaf\xdb\x8c\xd8\xb1\xd8\xa7', 1625921364.0), (b'\xd8\xab\xd8\xa8\xd8\xa7\xd8\xba', 1625921391.0), (b'\xd8\xab\xd8\xb1\xd9\x88\xd8\xaf', 1625921399.0), (b'\xd8\xb4\xd8\xaa\xd8\xb1\xd8\xa7\xd9\x86', 1625921399.0), (b'\xd8\xb4\xd8\xa8\xd8\xb1\xdb\x8c\xd8\xb2', 1625921425.0), (b'\xd8\xb4\xd8\xa8\xd9\x86\xd8\xaf\xd8\xb1', 1625921437.0), (b'\xd9\xbe\xd8\xaa\xd8\xb1\xd9\x88\xd9\x84', 1625921448.0), (b'\xd8\xa8\xd9\xbe\xdb\x8c\xd9\x88\xd9\x86\xd8\xaf', 1625921482.0), (b'\xd8\xb4\xd8\xa7\xd8\xae\xd8\xb5_\xd8\xa8\xd9\x88\xd8\xb1\xd8\xb3', 1625921489.0), (b'\xd8\xaf\xd8\xb2\xd9\x87\xd8\xb1\xd8\xa7\xd9\x88\xdb\x8c', 1625921497.0)]


In [71]:
tags_df = r.lrange('hashtags_recently_list', 0, -1)
tags_df = pd.DataFrame(tags_df)
tags_df[0] = tags_df[0].apply(lambda x : x.decode('utf-8'))
tags_df

,0
0,شپلی
1,وسپهر
2,غنوش
3,غپینو
4,گشان
...,...
995,زبینا
996,رکیش
997,فسا
998,دارا_یکم


In [ ]:
import pandas as pd

In [32]:
tmp = r.lrange('hashtags_recently_list', 0, -1)
str1 = b"\n ".join(tmp)
df = pd.DataFrame(tmp)
df[0] = df[0].apply(lambda x : x.decode('utf-8'))

In [50]:
df

,0
0,#وسینا دوستانی که با استراتژی ایچیموکو کار میک...
1,#سصوفی خداروشکر سصوفی امروز عالی بود و خدا بخ...
2,#وآیند شاخص ۳شنبه هفته قبل ۲۶۰۰۰مثبت بودووآیند...
3,#پخش در حال بهینه سازی زمین ها برای و املاک جه...
4,#شیران
...,...
95,#وآیند شاخص ۳شنبه هفته قبل ۲۶۰۰۰مثبت بودووآیند...
96,#پخش در حال بهینه سازی زمین ها برای و املاک جه...
97,#شیران
98,#ثرود \nبازگشایی با دامنه‌ی محدود نوسان هستش (...


In [ ]:
cc = 0
for s in tmp:
    cc+=1
    print(s.decode('utf-8'))
    if cc > 10 :
        break

In [ ]:
str1.decode('utf-8')

In [56]:
sample_twitt

{'id': '321799086',
 'sendTime': '2021-07-10T16:45:05Z',
 'sendTimePersian': '1400/04/19 21:15',
 'senderName': 'yr1isone',
 'senderUsername': 'yr1isone',
 'senderProfileImage': 'default',
 'content': '#رکیش امروز رو چه قیمتی می خواست باز بشه که باطلش کردن؟',
 'type': 'twit',
 'scoredPostDate': '1625935512874',
 'finalPullDatePersian': ''}

In [54]:
sample_twitt = {}

In [55]:
for message in consumer:
    sample_twitt = message.value
    break

